# IMPORT DES LIBRAIRIES 

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

# OUVERTURE DU JEU DE DONNEES

In [2]:
df = pd.read_csv('../DATA/OUT/new_dataset_with_data_prepocessing.csv')
df.head()

,IDVentes,Prix,Date,SurfaceTerrain,SurfaceBati,SurfaceCarrez,NombreLots,NombrePiecesPrincipales,CodePostal,TypeLocalName,slice,quarter,Year,Month,AvecTerrain,SurfaceBatiCat
0,174617,103950.0,2021-01-06,0,19,19.01,1,1,33600,Appartement,"[100000.0, 150000.0)",1,2021,1,False,"(0.999, 35.0]"
1,174618,214380.0,2021-01-05,0,38,38.57,2,1,33000,Appartement,"[200000.0, 250000.0)",1,2021,1,False,"(35.0, 47.0]"
2,174619,177000.0,2021-01-08,0,63,0.00,3,1,33320,Appartement,"[150000.0, 200000.0)",1,2021,1,False,"(60.0, 70.0]"
3,174620,153000.0,2021-01-08,0,30,0.00,2,1,33800,Appartement,"[150000.0, 200000.0)",1,2021,1,False,"(0.999, 35.0]"
4,174621,223170.0,2021-01-08,0,61,0.00,3,1,33127,Appartement,"[200000.0, 250000.0)",1,2021,1,False,"(60.0, 70.0]"


# CREATION D'UN ID PAR INDIVIDU

In [3]:
# Ajout d'une colonne ID qui sera un identifiant unique pour chaque ligne (commençant à 1)
df['ID'] = range(1, len(df) + 1)

# Vérification de l'ajout de la colonne ID
df[['ID'] + df.columns.tolist()[:-1]].head()

,ID,IDVentes,Prix,Date,SurfaceTerrain,SurfaceBati,SurfaceCarrez,NombreLots,NombrePiecesPrincipales,CodePostal,TypeLocalName,slice,quarter,Year,Month,AvecTerrain,SurfaceBatiCat
0,1,174617,103950.0,2021-01-06,0,19,19.01,1,1,33600,Appartement,"[100000.0, 150000.0)",1,2021,1,False,"(0.999, 35.0]"
1,2,174618,214380.0,2021-01-05,0,38,38.57,2,1,33000,Appartement,"[200000.0, 250000.0)",1,2021,1,False,"(35.0, 47.0]"
2,3,174619,177000.0,2021-01-08,0,63,0.00,3,1,33320,Appartement,"[150000.0, 200000.0)",1,2021,1,False,"(60.0, 70.0]"
3,4,174620,153000.0,2021-01-08,0,30,0.00,2,1,33800,Appartement,"[150000.0, 200000.0)",1,2021,1,False,"(0.999, 35.0]"
4,5,174621,223170.0,2021-01-08,0,61,0.00,3,1,33127,Appartement,"[200000.0, 250000.0)",1,2021,1,False,"(60.0, 70.0]"


# DEFINITION DES OUTLIERS POUR LES SURFACES DE TERRAINS PAR RAPPORT A LEUR PRIX

### Calcul D’Outliers :
X < q1 - 1.5 x iqr X > q3 + 1.5 x iqr

# CALCUL DU PRIX TERRAIN AU M²

In [4]:
# ISOLER LES BIENS AVEC TERRAINS
bien_avec_terrain = df.loc[(df['SurfaceTerrain'] > 0)]

df['PrixTerrainM2'] = df['Prix'] / df['SurfaceTerrain']
bien_avec_terrain['PrixTerrainM2'] = bien_avec_terrain['Prix'] / bien_avec_terrain['SurfaceTerrain']
# Remplacez les valeurs infinies par zéro.
df['PrixTerrainM2'].replace(np.inf, 0, inplace=True)
bien_avec_terrain['PrixTerrainM2'].replace(np.inf, 0, inplace=True)

C:\Users\jonat\AppData\Local\Temp\ipykernel_33168\3682938769.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bien_avec_terrain['PrixTerrainM2'] = bien_avec_terrain['Prix'] / bien_avec_terrain['SurfaceTerrain']
C:\Users\jonat\AppData\Local\Temp\ipykernel_33168\3682938769.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bien_avec_terrain['PrixTerrainM2'].replace(np.inf, 0, inplace=True)


# CALCULS DES QUARTILES POUR NOS OUTLIERS

In [5]:
# CALCUL DE NOTRE PREMIER QUARTILE
q1_prix_terrain = bien_avec_terrain['PrixTerrainM2'].quantile([.25])
q1_prix_terrain = q1_prix_terrain[0.25]
q3_prix_terrain = bien_avec_terrain['PrixTerrainM2'].quantile([.75])
q3_prix_terrain = q3_prix_terrain[0.75]

# CALCUL DE NOTRE ECART INTER QUARTILES
iqr_prix_terrain = q3_prix_terrain - q1_prix_terrain

print(f"q1 = {q1_prix_terrain}, q3 = {q3_prix_terrain}, IQR = {iqr_prix_terrain}")

q1 = 299.7525479616307, q3 = 1302.1973039215686, IQR = 1002.444755959938


# CALCUL DES VALEURS LIMITE OUTLIERS

In [6]:
outliers_prix_terrain_m2_inferieur = q1_prix_terrain - (1.5 * iqr_prix_terrain)
outliers_prix_terrain_m2_superieur = q3_prix_terrain + (1.5 * iqr_prix_terrain)

print(f'Outliers Inférieur = {outliers_prix_terrain_m2_inferieur}')
print(f'Outliers Supérieur = {outliers_prix_terrain_m2_superieur}')

Outliers Inférieur = -1203.9145859782764
Outliers Supérieur = 2805.8644378614754


# COMPATAGE DES OUTLIERS

In [7]:
nb_outliers_surface_terrain = df.loc[(df['PrixTerrainM2'] > outliers_prix_terrain_m2_superieur) | (df['PrixTerrainM2'] < outliers_prix_terrain_m2_inferieur)].shape
print(f'Nous aurions donc {nb_outliers_surface_terrain[0]} outliers.')

Nous aurions donc 13041 outliers.


# STOCKAGE DES OUTLIERS DANS UN NOUVEAU DATAFRAME

In [8]:
outliers_surface_terrain = df.loc[(df['PrixTerrainM2'] > outliers_prix_terrain_m2_superieur) | (df['PrixTerrainM2'] < outliers_prix_terrain_m2_inferieur)]

# DEFINITION DES OUTLIERS POUR LES SURFACES BATI PAR RAPPORT A LEUR PRIX
# CALCUL DU PRIX BATI AU M²

In [9]:
df['PrixBatiM2'] = df['Prix'] / df['SurfaceBati']

# CALCULS DES QUARTILES POUR NOS OUTLIERS

In [10]:
# CALCUL DE NOTRE PREMIER QUARTILE
q1_prix_bati = df['PrixBatiM2'].quantile([.25])
q1_prix_bati = q1_prix_bati[0.25]
q3_prix_bati = df['PrixBatiM2'].quantile([.75])
q3_prix_bati = q3_prix_bati[0.75]

# CALCUL DE NOTRE ECART INTER QUARTILES
iqr_prix_bati = q3_prix_bati - q1_prix_bati

print(f"q1 = {q1_prix_bati}, q3 = {q3_prix_bati}, IQR = {iqr_prix_bati}")

q1 = 2335.770676691729, q3 = 5000.0, IQR = 2664.229323308271


# CALCUL DES VALEURS LIMITE OUTLIERS

In [11]:
outliers_prix_bati_m2_inferieur = q1_prix_bati - (1.5 * iqr_prix_bati)
outliers_prix_bati_m2_superieur = q3_prix_bati + (1.5 * iqr_prix_bati)

print(f'Outliers Inférieur = {outliers_prix_bati_m2_inferieur}')
print(f'Outliers Supérieur = {outliers_prix_bati_m2_superieur}')

Outliers Inférieur = -1660.5733082706774
Outliers Supérieur = 8996.343984962406


# COMPATAGE DES OUTLIERS

In [12]:
nb_outliers_surface_bati = df.loc[(df['PrixBatiM2'] > outliers_prix_bati_m2_superieur) | (df['PrixBatiM2'] < outliers_prix_bati_m2_inferieur)].shape
print(f'Nous aurions donc {nb_outliers_surface_bati[0]} outliers.')

Nous aurions donc 17989 outliers.


# STOCKAGE DES OUTLIERS DANS UN NOUVEAU DATAFRAME

In [13]:
outliers_surface_bati = df.loc[(df['PrixBatiM2'] > outliers_prix_bati_m2_superieur) | (df['PrixBatiM2'] < outliers_prix_bati_m2_inferieur)]

# DEFINITION DES OUTLIERS POUR LES SURFACES CARREZ PAR RAPPORT A LEUR PRIX
# CALCUL DU PRIX CARREZ AU M²

In [14]:
# ISOLER LES BIENS AVEC UNE SURFACE CARREZ
bien_avec_carrez = df.loc[(df['SurfaceCarrez'] > 0)]

df['PrixCarrezM2'] = df['Prix'] / df['SurfaceCarrez']
bien_avec_carrez['PrixCarrezM2'] = bien_avec_carrez['Prix'] / bien_avec_carrez['SurfaceCarrez']
# Remplacez les valeurs infinies par zéro.
df['PrixCarrezM2'].replace(np.inf, 0, inplace=True)
bien_avec_carrez['PrixCarrezM2'].replace(np.inf, 0, inplace=True)

C:\Users\jonat\AppData\Local\Temp\ipykernel_33168\3278027763.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bien_avec_carrez['PrixCarrezM2'] = bien_avec_carrez['Prix'] / bien_avec_carrez['SurfaceCarrez']
C:\Users\jonat\AppData\Local\Temp\ipykernel_33168\3278027763.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bien_avec_carrez['PrixCarrezM2'].replace(np.inf, 0, inplace=True)


# CALCULS DES QUARTILES POUR NOS OUTLIERS

In [15]:
# CALCUL DE NOTRE PREMIER QUARTILE
q1_prix_carrez = bien_avec_carrez['PrixCarrezM2'].quantile([.25])
q1_prix_carrez = q1_prix_carrez[0.25]
q3_prix_carrez = bien_avec_carrez['PrixCarrezM2'].quantile([.75])
q3_prix_carrez = q3_prix_carrez[0.75]

# CALCUL DE NOTRE ECART INTER QUARTILES
iqr_prix_carrez = q3_prix_carrez - q1_prix_carrez

print(f"q1 = {q1_prix_carrez}, q3 = {q3_prix_carrez}, IQR = {iqr_prix_carrez}")

q1 = 2829.9893192261097, q3 = 4811.123744407223, IQR = 1981.1344251811133


# CALCUL DES VALEURS LIMITE OUTLIERS

In [16]:
outliers_prix_carrez_m2_inferieur = q1_prix_carrez - (1.5 * iqr_prix_carrez)
outliers_prix_carrez_m2_superieur = q3_prix_carrez + (1.5 * iqr_prix_carrez)

print(f'Outliers Inférieur = {outliers_prix_carrez_m2_inferieur}')
print(f'Outliers Supérieur = {outliers_prix_carrez_m2_superieur}')

Outliers Inférieur = -141.71231854556027
Outliers Supérieur = 7782.825382178893


# COMPATAGE DES OUTLIERS

In [17]:
nb_outliers_surface_carrez = df.loc[(df['PrixCarrezM2'] > outliers_prix_carrez_m2_superieur) | (df['PrixCarrezM2'] < outliers_prix_carrez_m2_inferieur)].shape
print(f'Nous aurions donc {nb_outliers_surface_carrez[0]} outliers.')

Nous aurions donc 1379 outliers.


# STOCKAGE DES OUTLIERS DANS UN NOUVEAU DATAFRAME

In [18]:
outliers_surface_carrez = df.loc[(df['PrixCarrezM2'] > outliers_prix_carrez_m2_superieur) | (df['PrixCarrezM2'] < outliers_prix_carrez_m2_inferieur)]

# COMPATEGE DU TOTAL D'OUTLIERS DEPUIS LE DEBUT (DOUBLONS INCLUS)

In [19]:
nb_outliers_surface_terrain[0] + nb_outliers_surface_bati[0] + nb_outliers_surface_carrez[0]

32409

# CREATION D'UN DATAFRAME HORS OUTLIERS ET DOUBLONS D'OUTLIERS

In [20]:
# Concaténation des ID des outliers
ids_outliers = pd.concat([outliers_surface_bati['ID'], outliers_surface_terrain['ID'], outliers_surface_carrez['ID']])

# Suppression des duplicatas
ids_outliers = ids_outliers.drop_duplicates()

# Filtrage de df pour exclure les outliers
df_clean = df[~df['ID'].isin(ids_outliers)]

# CREATION D'UN CSV DEPUIS CE DATAFRAMES

In [21]:
df_clean.to_csv('../DATA/OUT/new_dataset_hors_outliers_surfaces.csv', index=False)

# DEFINITION DES OUTLIERS POUR LE NOMBRE DE PIECES PRINCIPALES

# CALCULS DES QUARTILES POUR NOS OUTLIERS

In [22]:
# CALCUL DE NOTRE PREMIER QUARTILE
q1_nb_piece = df['NombrePiecesPrincipales'].quantile([.25])
q1_nb_piece = q1_nb_piece[0.25]
q3_nb_piece = df['NombrePiecesPrincipales'].quantile([.75])
q3_nb_piece = q3_nb_piece[0.75]

# CALCUL DE NOTRE ECART INTER QUARTILES
iqr_nb_piece = q3_nb_piece - q1_nb_piece

print(f"q1 = {q1_nb_piece}, q3 = {q3_nb_piece}, IQR = {iqr_nb_piece}")

q1 = 0.0, q3 = 1.0, IQR = 1.0


# CALCUL DES VALEURS LIMITE OUTLIERS

In [23]:
outliers_nb_pieces_inferieur = q1_nb_piece - (1.5 * iqr_nb_piece)
outliers_nb_pieces_superieur = q3_nb_piece + (1.5 * iqr_nb_piece)

print(f'Outliers Inférieur = {outliers_nb_pieces_inferieur}')
print(f'Outliers Supérieur = {outliers_nb_pieces_superieur}')

Outliers Inférieur = -1.5
Outliers Supérieur = 2.5


# COMPATAGE DES OUTLIERS

In [24]:
nb_outliers_nb_piece = df.loc[(df['NombrePiecesPrincipales'] > outliers_nb_pieces_superieur) | (df['NombrePiecesPrincipales'] < outliers_nb_pieces_inferieur)].shape
print(f'Nous aurions donc {nb_outliers_nb_piece[0]} outliers.')

Nous aurions donc 1664 outliers.


# STOCKAGE DES OUTLIERS DANS UN NOUVEAU DATAFRAME

In [25]:
outliers_nb_pieces = df.loc[(df['NombrePiecesPrincipales'] > outliers_nb_pieces_superieur) | (df['NombrePiecesPrincipales'] < outliers_nb_pieces_inferieur)]

# CREATION D'UN DATAFRAME HORS OUTLIERS ET DOUBLONS D'OUTLIERS

In [26]:
# Concaténation des ID des outliers
ids_outliers = pd.concat([outliers_surface_bati['ID'], outliers_surface_terrain['ID'], outliers_surface_carrez['ID'], outliers_nb_pieces['ID']])

# Suppression des duplicatas
ids_outliers = ids_outliers.drop_duplicates()

# Filtrage de df pour exclure les outliers
df_clean_with_nb_piece = df[~df['ID'].isin(ids_outliers)]

# CREATION D'UN CSV DEPUIS CE DATAFRAMES

In [27]:
df_clean_with_nb_piece.to_csv('../DATA/OUT/new_dataset_hors_outliers_surfaces_and_piece.csv', index=False)